# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}
fields = {'BusinessName': 1, 'scores.Hygiene': 1}

# Use count_documents to display the number of documents in the result
document_count = establishments.count_documents(query)
print("Total document count:", document_count)

# Display the first document in the results using pprint
results = list(establishments.find(query, fields))
pprint(results[0])

Total document count: 41
{'BusinessName': 'The Chase Rest Home',
 '_id': ObjectId('669d3b8dd6fa6b5ecafe190c'),
 'scores': {'Hygiene': 20}}


In [7]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(result_df))

# Display the first 10 rows of the DataFrame
result_df.head(10)

Rows in DataFrame:  41


,_id,BusinessName,scores
0,669d3b8dd6fa6b5ecafe190c,The Chase Rest Home,{'Hygiene': 20}
1,669d3b8dd6fa6b5ecafe1c8f,Brenalwood,{'Hygiene': 20}
2,669d3b8dd6fa6b5ecafe1f9a,Melrose Hotel,{'Hygiene': 20}
3,669d3b8dd6fa6b5ecafe2188,Seaford Pizza,{'Hygiene': 20}
4,669d3b8dd6fa6b5ecafe2196,Golden Palace,{'Hygiene': 20}
5,669d3b8ed6fa6b5ecafe2b3a,Ashby's Butchers,{'Hygiene': 20}
6,669d3b8ed6fa6b5ecafe2d54,South Sea Express Cuisine,{'Hygiene': 20}
7,669d3b8ed6fa6b5ecafe4280,Golden Palace,{'Hygiene': 20}
8,669d3b8ed6fa6b5ecafe46d8,The Tulip Tree,{'Hygiene': 20}
9,669d3b8ed6fa6b5ecafe4ed8,F & S,{'Hygiene': 20}


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query =

# Use count_documents to display the number of documents in the result

# Display the first document in the results using pprint


In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude =
longitude =

query =
sort =
limit =

# Print the results


In [ ]:
# Convert result to Pandas DataFrame


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results


In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame
